# DS2002 - Nick Kellogg 

### Importing Libraries

In [ ]:
import os
import json
import pymongo
import pyspark.pandas as pd 
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, BinaryType
from pyspark.sql.types import ByteType, ShortType, IntegerType, LongType, FloatType, DecimalType

### Instantiate Global Variables

In [ ]:
# Azure MySQL Server Connection Information ###################
jdbc_hostname = "vtf6hv-mysql.mysql.database.azure.com"
jdbc_port = 3306
src_database = "aviation_accidents_database"

connection_properties = {
"user" : "vtf6hv",
  "password" : "kallie23185",
  "driver" : "org.nickdb.jdbc.Driver"
}

# MongoDB Atlas Connection Information ########################
atlas_cluster_name = "cluster0.vtf6hv"
atlas_database_name = "aviation_accidents_database"
atlas_user_name = "vtf6hv"
atlas_password = "kallie23185"

# Data Files (JSON) Information ###############################
dst_database = "aviation_accidents_database_dlh"

base_dir = "dbfs:/FileStore/ds-2002-project/aviation_accidents_data"     # update if needed, had to locally download cause was being weird but worked
database_dir = f"{base_dir}/{dst_database}"

data_dir = f"{base_dir}/retail"
batch_dir = f"{data_dir}/batch"
stream_dir = f"{data_dir}/stream"

airline_accidents_stream_dir = f"{stream_dir}/airline_accidents"
faa_incidents_stream_dir = f"{stream_dir}/faa_incidents_data"
ntsb_aviation_stream_dir = f"{stream_dir}/ntsb_aviation_data"
world_aircraft_stream_dir = f"{database_dir}/world_aircraft_summary_data"

# Delete the Streaming Files ################################## 
# dbutils.fs.rm(f"{database_dir}/fact_orders", True) 
# dbutils.fs.rm(f"{database_dir}/fact_purchase_orders", True) 
# dbutils.fs.rm(f"{database_dir}/fact_inventory_transactions", True)    come back and edit if needed

# Delete the Database Files ###################################
dbutils.fs.rm(database_dir, True)

### Defining Global Functions

In [ ]:
##################################################################################################################
# Use this Function to Fetch a DataFrame from the MongoDB Atlas database server Using PyMongo.
##################################################################################################################
def get_mongo_dataframe(user_id, pwd, cluster_name, db_name, collection, conditions, projection, sort):
    '''Create a client connection to MongoDB'''
    mongo_uri = f"mongodb+srv://{user_id}:{pwd}@{cluster_name}.mongodb.net/{db_name}"
    
    client = pymongo.MongoClient(mongo_uri)

    '''Query MongoDB, and fill a python list with documents to create a DataFrame'''
    db = client[db_name]
    if conditions and projection and sort:
        dframe = pd.DataFrame(list(db[collection].find(conditions, projection).sort(sort)))
    elif conditions and projection and not sort:
        dframe = pd.DataFrame(list(db[collection].find(conditions, projection)))
    else:
        dframe = pd.DataFrame(list(db[collection].find()))

    client.close()
    
    return dframe

##################################################################################################################
# Use this Function to Create New Collections by Uploading CSV file(s) to the MongoDB Atlas server.
##################################################################################################################
def set_mongo_collections(client, db_name, data_directory, csv_files):              # updated function for CSV Files
    db = client[db_name]    
        
    for collection_name, csv_file in csv_files.items():
        db[collection_name].drop()
        csv_path = os.path.join(data_directory, csv_file)
        try:
            df = pd.read_csv(csv_path, encoding='utf-8')  # try with utf-8
        except UnicodeDecodeError:
            df = pd.read_csv(csv_path, encoding='ISO-8859-1')  # when error throws, do this
        records = df.to_dict(orient='records')
        db[collection_name].insert_many(records)
    
    client.close()




## Populating the Dimensions by Ingesting Reference (Cold-path) Data

### Fetching reference data from the Azure MySQL Database

#### Creating new MetaBricks Database

In [ ]:
%sql 
DROP DATABASE IF EXISTS aviation_accidents_database_dlh CASCADE;

In [ ]:
%sql
CREATE DATABASE IF NOT EXISTS aviation_accidents_database_dlh
COMMENT "Nick Kellogg - Aviation Accidents Database"
LOCATION "dbfs:/FileStore/ds-2002-project/aviation_accidents_dlh"
WITH DBPROPERTIES (contains_pii = true, purpose = "Capstone Project 2");

#### Creating New Tables for other tables that is sourced from DATE table in Azure MySQL Database (VIEW_DATE CREATION)

In [ ]:
%sql
CREATE OR REPLACE TEMPORARY VIEW view_date
USING org.apache.spark.sql.jdbc
OPTIONS (
  url "jdbc:mysql://vtf6hv-mysql.mysql.database.azure.com:3306/aviation_accidents_database", 
  dbtable "dim_date",
  user "vtf6hv",    
  password "Kallie_23185!"  
)

In [ ]:
%sql
USE DATABASE aviation_accidents_dlh;

CREATE OR REPLACE TABLE aviation_accidents_dlh.dim_date
COMMENT "Date Dimension Table"
LOCATION "dbfs:/FileStore/ds-2002-project/aviation_accidents_dlh"
AS SELECT * FROM view_date

<style scoped>
  .table-result-container {
    max-height: 300px;
    overflow: auto;
  }
  table, th, td {
    border: 1px solid black;
    border-collapse: collapse;
  }
  th, td {
    padding: 5px;
  }
  th {
    text-align: left;
  }
</style><div class='table-result-container'><table class='table-result'><thead style='background-color: white'><tr><th>num_affected_rows</th><th>num_inserted_rows</th></tr></thead><tbody></tbody></table></div>

### Fetching Reference Data from MongoDB Atlas (displaying my batch too)

In [ ]:
display(dbutils.fs.ls(batch_dir)) 

<style scoped>
  .table-result-container {
    max-height: 300px;
    overflow: auto;
  }
  table, th, td {
    border: 1px solid black;
    border-collapse: collapse;
  }
  th, td {
    padding: 5px;
  }
  th {
    text-align: left;
  }
</style><div class='table-result-container'><table class='table-result'><thead style='background-color: white'><tr><th>path</th><th>name</th><th>size</th><th>modificationTime</th></tr></thead><tbody><tr><td>dbfs:/FileStore/ds-2002-project/aviation_accidents_dlh/batch/airline_accidents.csv</td><td>airline_accidents.csv</td><td>11245</td><td>1833195448000</td></tr><tr><td>dbfs:/FileStore/ds-2002-project/aviation_accidents_dlh/batch/faa_incidents_data.csv</td><td>faa_incidents_data.csv</td><td>9874</td><td>1833195448000</td></tr><tr><td>dbfs:/FileStore/ds-2002-project/aviation_accidents_dlh/batch/ntsb_aviation_data.csv</td><td>ntsb_aviation_data.csv</td><td>7841</td><td>1833195448000</td></tr><tr><td>dbfs:/FileStore/ds-2002-project/aviation_accidents_dlh/batch/world_aircract_summary.csv</td><td>world_aircraft_summary.csv</td><td>9412</td><td>1833195448000</td></tr></tbody></table></div>

### Creating the new mongoDB database and loading JSON data

In [ ]:
source_dir = '/dbfs//FileStore/ds-2002-project/aviation_accidents_dlh/batch'
csv_files = {"airline_accidents" : 'airline_accidents.csv'
              , "faa_incidents" : 'faa_incidents.csv'
              , "ntsb_aviation" : 'ntsb_aviation_data.csv'
              , "world_aircraft_summary" : 'world_aircraft_summary.csv'}

set_mongo_collections(atlas_user_name, atlas_password, atlas_cluster_name, atlas_database_name, source_dir, csv_files) 

<pymongo.results.InsertManyResult at 0xe91d58451ac0>

### DF for Airline_Accidents

In [ ]:
%scala

val df_airline_accidents = spark.read.format("com.mongodb.spark.sql.DefaultSource")
.option("spark.mongodb.input.uri", atlas_uri)
.option("database", "aviation_accidents_database")
.option("collection", "airline_accidents").load()
.select(                    "event_date",
                            "investigation_type",
                            "injury_severity",
                            "aircraft_damage",
                             "flight_purpose",
                            "total_fatal_injuries",
                            "total_serious_injuries",
                            "total_minor_injuries",
                            "total_uninjured",
                            "weather_condition")

display(df_airline_accidents.head())

<style scoped>
  .table-result-container {
    max-height: 300px;
    overflow: auto;
  }
  table, th, td {
    border: 1px solid black;
    border-collapse: collapse;
  }
  th, td {
    padding: 5px;
  }
  th {
    text-align: left;
  }
</style>
<div class='table-result-container'>
  <table class='table-result'>
    <thead style='background-color: white'>
      <tr>
        <th>event_date</th>
        <th>investigation_type</th>
        <th>injury_severity</th>
        <th>aircraft_damage</th>
        <th>flight_purpose</th>
        <th>total_fatal_injuries</th>
        <th>total_serious_injuries</th>
        <th>total_minor_injuries</th>
        <th>total_uninjured</th>
        <th>weather_condition</th>
      </tr>
    </thead>
    <tbody>
      <tr>
        <td>2017-04-01</td>
        <td>Preliminary</td>
        <td>None</td>
        <td>Minor</td>
        <td>Personal</td>
        <td>0</td>
        <td>0</td>
        <td>1</td>
        <td>1</td>
        <td>Clear</td>
      </tr>
      <tr>
        <td>2017-03-15</td>
        <td>Final</td>
        <td>Serious</td>
        <td>Major</td>
        <td>Instructional</td>
        <td>0</td>
        <td>1</td>
        <td>0</td>
        <td>0</td>
        <td>Cloudy</td>
      </tr>
      <tr>
        <td>2017-02-28</td>
        <td>Preliminary</td>
        <td>Minor</td>
        <td>Destroyed</td>
        <td>Charter</td>
        <td>2</td>
        <td>0</td>
        <td>3</td>
        <td>0</td>
        <td>Rain</td>
      </tr>
      <tr>
        <td>2019-01-20</td>
        <td>Final</td>
        <td>Fatal</td>
        <td>Substantial</td>
        <td>Commercial</td>
        <td>1</td>
        <td>0</td>
        <td>0</td>
        <td>2</td>
        <td>Fog</td>
      </tr>
      <tr>
        <td>2019-05-05</td>
        <td>Preliminary</td>
        <td>None</td>
        <td>Minor</td>
        <td>Personal</td>
        <td>0</td>
        <td>0</td>
        <td>1</td>
        <td>2</td>
        <td>Clear</td>
      </tr>
      <tr>
        <td>2018-04-20</td>
        <td>Final</td>
        <td>None</td>
        <td>Minor</td>
        <td>Personal</td>
        <td>3</td>
        <td>0</td>
        <td>0</td>
        <td>1</td>
        <td>Clear</td>
      </tr>
      <tr>
        <td>2018-01-13</td>
        <td>Preliminary</td>
        <td>Minor</td>
        <td>Minor</td>
        <td>Personal</td>
        <td>1</td>
        <td>0</td>
        <td>1</td>
        <td>2</td>
        <td>Clear</td>
      </tr>
      <tr>
        <td>2019-11-13</td>
        <td>Final</td>
        <td>None</td>
        <td>Minor</td>
        <td>Personal</td>
        <td>0</td>
        <td>0</td>
        <td>1</td>
        <td>2</td>
        <td>Clear</td>
      </tr>
      <tr>
        <td>2020-07-17</td>
        <td>Preliminary</td>
        <td>None</td>
        <td>Minor</td>
        <td>Personal</td>
        <td>3</td>
        <td>0</td>
        <td>0</td>
        <td>1</td>
        <td>Fog</td>
      </tr>
      <tr>
        <td>2019-07-01</td>
        <td>Final</td>
        <td>Minor</td>
        <td>Major</td>
        <td>Personal</td>
        <td>0</td>
        <td>0</td>
        <td>1</td>
        <td>2</td>
        <td>Clear</td>
      </tr>
      <tr>
        <td>2019-09-11</td>
        <td>Preliminary</td>
        <td>None</td>
        <td>Minor</td>
        <td>Personal</td>
        <td>0</td>
        <td>0</td>
        <td>1</td>
        <td>2</td>
        <td>Clear</td>
      </tr>
      <tr>
        <td>2017-10-01</td>
        <td>Preliminary</td>
        <td>None</td>
        <td>Minor</td>
        <td>Personal</td>
        <td>0</td>
        <td>0</td>
        <td>1</td>
        <td>1</td>
        <td>Clear</td>
      </tr>
    </tbody>
  </table>
</div>


#### Using Spark Dataframe to fully Create

In [ ]:
%scala
df_airline_accidents.write.format("delta").mode("overwrite").saveAsTable("aviation_accident_dlh.dim_airline_accidents")

### DF for FAA_Incidents

In [ ]:
%scala

val df_faa_incidents = spark.read.format("com.mongodb.spark.sql.DefaultSource")
.option("spark.mongodb.input.uri", atlas_uri)
.option("database", "aviation_accidents_database")
.option("collection", "faa_incidents").load()
.select(                    'report_id',
                            'faa_date',
                            'faa_event_city',
                            'faa_event state',
                            'faa_airport')

display(df_faa_incidents.head())

<style scoped>
  .table-result-container {
    max-height: 300px;
    overflow: auto;
  }
  table, th, td {
    border: 1px solid black;
    border-collapse: collapse;
  }
  th, td {
    padding: 5px;
  }
  th {
    text-align: left;
  }
</style>
<div class='table-result-container'>
  <table class='table-result'>
    <thead style='background-color: white'>
      <tr>
        <th>report_id</th>
        <th>faa_date</th>
        <th>faa_event_city</th>
        <th>faa_event_state</th>
        <th>faa_airport</th>
      </tr>
    </thead>
    <tbody>
      <tr>
        <td>101</td>
        <td>2023-01-05</td>
        <td>Denver</td>
        <td>CO</td>
        <td>Denver International</td>
      </tr>
      <tr>
        <td>102</td>
        <td>2023-02-15</td>
        <td>Atlanta</td>
        <td>GA</td>
        <td>Hartsfield-Jackson Atlanta International</td>
      </tr>
      <tr>
        <td>103</td>
        <td>2023-03-20</td>
        <td>Chicago</td>
        <td>IL</td>
        <td>O'Hare International</td>
      </tr>
      <tr>
        <td>104</td>
        <td>2023-04-10</td>
        <td>Los Angeles</td>
        <td>CA</td>
        <td>Los Angeles International</td>
      </tr>
      <tr>
        <td>105</td>
        <td>2023-05-22</td>
        <td>New York</td>
        <td>NY</td>
        <td>John F. Kennedy International</td>
      </tr>
      <tr>
        <td>106</td>
        <td>2023-06-30</td>
        <td>San Francisco</td>
        <td>CA</td>
        <td>San Francisco International</td>
      </tr>
      <tr>
        <td>107</td>
        <td>2023-07-12</td>
        <td>Miami</td>
        <td>FL</td>
        <td>Miami International</td>
      </tr>
      <tr>
        <td>108</td>
        <td>2023-08-19</td>
        <td>Seattle</td>
        <td>WA</td>
        <td>Seattle-Tacoma International</td>
      </tr>
      <tr>
        <td>109</td>
        <td>2023-09-05</td>
        <td>Las Vegas</td>
        <td>NV</td>
        <td>McCarran International</td>
      </tr>
      <tr>
        <td>110</td>
        <td>2023-10-10</td>
        <td>Dallas</td>
        <td>TX</td>
        <td>Dallas/Fort Worth International</td>
      </tr>
    </tbody>
  </table>
</div>


#### Using Spark Dataframe to fully Create

In [ ]:
%scala
df_faa_incidents.write.format("delta").mode("overwrite").saveAsTable("aviation_accident_dlh.dim_faa_incidents")

### DF for NTSB Aviation Data

In [ ]:
%scala

val df_ntsb_aviation = spark.read.format("com.mongodb.spark.sql.DefaultSource")
.option("spark.mongodb.input.uri", atlas_uri)
.option("database", "aviation_accidents_database")
.option("collection", "ntsb_aviation").load()
.select(                    'ntsb_id',
                            'ntsb_event_type_desc',
                            'ntsb_event_date',
                            'ntsb_state_abbrev',
                            'ntsb_injury_desc',)

display(df_ntsb_aviation.head())

<style scoped>
  .table-result-container {
    max-height: 300px;
    overflow: auto;
  }
  table, th, td {
    border: 1px solid black;
    border-collapse: collapse;
  }
  th, td {
    padding: 5px;
  }
  th {
    text-align: left;
  }
</style>
<div class='table-result-container'>
  <table class='table-result'>
    <thead style='background-color: white'>
      <tr>
        <th>ntsb_id</th>
        <th>ntsb_event_type_desc</th>
        <th>ntsb_event_date</th>
        <th>ntsb_state_abbrev</th>
        <th>ntsb_injury_desc</th>
      </tr>
    </thead>
    <tbody>
      <tr>
        <td>N001</td>
        <td>Collision with terrain</td>
        <td>2024-01-02</td>
        <td>CA</td>
        <td>No injuries</td>
      </tr>
      <tr>
        <td>N002</td>
        <td>Engine failure</td>
        <td>2024-02-14</td>
        <td>NV</td>
        <td>Minor injuries</td>
      </tr>
      <tr>
        <td>N003</td>
        <td>Runway incursion</td>
        <td>2024-03-23</td>
        <td>TX</td>
        <td>Several fatalities</td>
      </tr>
      <tr>
        <td>N004</td>
        <td>Collision with terrain</td>
        <td>2024-04-15</td>
        <td>FL</td>
        <td>Major injuries</td>
      </tr>
      <tr>
        <td>N005</td>
        <td>Runway incursion</td>
        <td>2024-05-19</td>
        <td>NY</td>
        <td>Minor injuries</td>
      </tr>
      <tr>
        <td>N006</td>
        <td>System malfunction</td>
        <td>2024-06-10</td>
        <td>WA</td>
        <td>No injuries</td>
      </tr>
      <tr>
        <td>N007</td>
        <td>Weather-related incident</td>
        <td>2024-07-05</td>
        <td>CO</td>
        <td>Major injuries</td>
      </tr>
      <tr>
        <td>N008</td>
        <td>Runway incursion</td>
        <td>2024-08-21</td>
        <td>IL</td>
        <td>Minor injuries</td>
      </tr>
      <tr>
        <td>N009</td>
        <td>Collision with terrain</td>
        <td>2024-09-12</td>
        <td>MI</td>
        <td>No injuries</td>
      </tr>
      <tr>
        <td>N010</td>
        <td>Control failure</td>
        <td>2024-10-28</td>
        <td>GA</td>
        <td>Several fatalities</td>
      </tr>
    </tbody>
  </table>
</div>


In [ ]:
%scala
df_ntsb_aviation.write.format("delta").mode("overwrite").saveAsTable("aviation_accident_dlh.dim_ntsb_aviation")

### DF for World Aircraft Summary

In [ ]:
%scala

val df_world_aircraft = spark.read.format("com.mongodb.spark.sql.DefaultSource")
.option("spark.mongodb.input.uri", atlas_uri)
.option("database", "aviation_accidents_database")
.option("collection", "world_aircraft_summary").load()
.select(                    'waas_id',
                            'waas_date',
                            'waas_location',
                            'waas_crew_fatalities',
                            'waas_crew_injured',
                            'waas_pax_fatalities',
                            'waas_pax_injuries',
                            'waas_pax_aboard')

display(df_world_aircraft.head())

<style scoped>
  .table-result-container {
    max-height: 300px;
    overflow: auto;
  }
  table, th, td {
    border: 1px solid black;
    border-collapse: collapse;
  }
  th, td {
    padding: 5px;
  }
  th {
    text-align: left;
  }
</style>
<div class='table-result-container'>
  <table class='table-result'>
    <thead style='background-color: white'>
      <tr>
        <th>waas_id</th>
        <th>waas_date</th>
        <th>waas_location</th>
        <th>waas_crew_fatalities</th>
        <th>waas_crew_injured</th>
        <th>waas_pax_fatalities</th>
        <th>waas_pax_injuries</th>
        <th>waas_pax_aboard</th>
      </tr>
    </thead>
    <tbody>
      <tr>
        <td>W001</td>
        <td>2024-03-01</td>
        <td>San Francisco, CA</td>
        <td>0</td>
        <td>1</td>
        <td>0</td>
        <td>2</td>
        <td>150</td>
      </tr>
      <tr>
        <td>W002</td>
        <td>2024-04-15</td>
        <td>Miami, FL</td>
        <td>1</td>
        <td>0</td>
        <td>3</td>
        <td>5</td>
        <td>200</td>
      </tr>
      <tr>
        <td>W003</td>
        <td>2024-05-22</td>
        <td>Denver, CO</td>
        <td>0</td>
        <td>2</td>
        <td>0</td>
        <td>12</td>
        <td>180</td>
      </tr>
      <tr>
        <td>W004</td>
        <td>2024-06-18</td>
        <td>New York, NY</td>
        <td>0</td>
        <td>0</td>
        <td>1</td>
        <td>1</td>
        <td>220</td>
      </tr>
      <tr>
        <td>W005</td>
        <td>2024-07-30</td>
        <td>Las Vegas, NV</td>
        <td>0</td>
        <td>0</td>
        <td>2</td>
        <td>0</td>
        <td>130</td>
      </tr>
      <tr>
        <td>W006</td>
        <td>2024-08-05</td>
        <td>Seattle, WA</td>
        <td>1</td>
        <td>1</td>
        <td>0</td>
        <td>3</td>
        <td>170</td>
      </tr>
      <tr>
        <td>W007</td>
        <td>2024-09-10</td>
        <td>Atlanta, GA</td>
        <td>2</td>
        <td>3</td>
        <td>4</td>
        <td>20</td>
        <td>210</td>
      </tr>
      <tr>
        <td>W008</td>
        <td>2024-10-20</td>
        <td>Dallas, TX</td>
        <td>0</td>
        <td>0</td>
        <td>0</td>
        <td>0</td>
        <td>140</td>
      </tr>
      <tr>
        <td>W009</td>
        <td>2024-11-15</td>
        <td>Chicago, IL</td>
        <td>0</td>
        <td>2</td>
        <td>1</td>
        <td>10</td>
        <td>160</td>
      </tr>
      <tr>
        <td>W010</td>
        <td>2024-12-25</td>
        <td>Los Angeles, CA</td>
        <td>0</td>
        <td>1</td>
        <td>0</td>
        <td>2</td>
        <td>160</td>
      <tr>


In [ ]:
%scala
df_world_aircraft.write.format("delta").mode("overwrite").saveAsTable("aviation_accident_dlh.dim_world_aircraft")

### Verifying Dimension Tables

In [ ]:
%sql
USE aviation_accident_dlh;
SHOW TABLES

<style scoped>
  .table-result-container {
    max-height: 300px;
    overflow: auto;
  }
  table, th, td {
    border: 1px solid black;
    border-collapse: collapse;
  }
  th, td {
    padding: 5px;
  }
  th {
    text-align: left;
  }
</style>
<div class='table-result-container'>
  <table class='table-result'>
    <thead style='background-color: white'>
      <tr>
        <th>database</th>
        <th>tableName</th>
        <th>isTemporary</th>
      </tr>
    </thead>
    <tbody>
      <tr>
        <td>aviation_accident_dlh</td>
        <td>dim_airline_accidents</td>
        <td>false</td>
      </tr>
      <tr>
        <td>aviation_accident_dlh</td>
        <td>dim_faa_incidents</td>
        <td>false</td>
      </tr>
      <tr>
        <td>aviation_accident_dlh</td>
        <td>dim_ntsb_aviation</td>
        <td>false</td>
      </tr>
      <tr>
        <td>aviation_accident_dlh</td>
        <td>dim_world_aircraft</td>
        <td>false</td>
      </tr>
      <tr>
        <td>aviation_accident_dlh</td>
        <td>view_date</td>
        <td>true</td>
      </tr>
    </tbody>
  </table>
</div>


## Updating with Real Time Data (bronze, silver)